# Project Objective

**Context**: The company is looking at a few regions to start new oil wells.

**Objective**: Our objective is to find the most lucrative region for the new oil wells. For that, we'll build a Machine Learning model that can predict the volume of the reserves in the oil wells.

**Data Description**

`geo_data_0, 1 and 2` - datasets, each representing a different region.
 
`id` - unique oil well identifier.

`f0, f1, f2` - oil well characteristics.

`product` - volume of reserves in the oil well (thousands of barrels).

**Overall Conditions**:
 - We'll use a linear regression model to train the machine.
 
 - When a region is studied, 500 points are analysed and the best 200 are selected to calculate profits.
 
 - The budget we have to develop 200 oil wells is USD 100M.
 
 - One oil barrel brings USD 4.5 in revenues. One unit of product brings USD 4.5k (1 unit = 1k barrels).
 
 - We'll filter out any regions with a loss risk higher than 2.5%, and between the ones left, we'll pick the one with the highest average profit.

## 1 Loading Data and Libraries, preparing data

In this section of the project, we'll:
 - load the necessary libraries 
 - and also load and prepare the data
     - see if there is missing data, invalid data, 
     - and also encode categoricals, remove unnecessary columns, scale numericals
     
We'll basically get the dataset ready for our ML training.

In [1]:
#loading libraries

import pandas as pd
import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
import math
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error



In [2]:
#loading data

#geo_data_0 = pd.read_csv(r'C:\temptripleten\geo_data_0.csv')
#geo_data_1 = pd.read_csv(r'C:\temptripleten\geo_data_1.csv')
#geo_data_2 = pd.read_csv(r'C:\temptripleten\geo_data_2.csv')

geo_data_0 = pd.read_csv(r'/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv(r'/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv(r'/datasets/geo_data_2.csv')

In [3]:
#quick snapshot of the datasets

dfs_list_names = ['geo_data_0', 'geo_data_1', 'geo_data_2']
dfs_list = [geo_data_0, geo_data_1, geo_data_2]

for df_name, df in zip(dfs_list_names, dfs_list):
    print(df_name)
    display(df.head(3))
    display(df.info())
    display(df.describe())

geo_data_0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


...

geo_data_2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


**Summary**

It seems the datasets are in order - no missing values, correct data types for the columns, and the values seem to be scaled already, for the features (f0, f1, f2).

We'll proceed by splitting the dataset between features and targets, then by training and testing.

## 2 Creating and Testing the Model

In this section, we aim to achieve the following:

 1. Split the datasets between features and targets,
 
 2. Split the datasets between training and testing/validation (75%, 25%),
 
 3. Train the model and make predictions on the test datasets,
 
 4. Print the average product volume predicted and the model's RMSE,
 
 5. Analyse Results.
 
Since we'll repeat the same processes for all 3 datasets, we'll produce functions during the modeling of the first dataset, so we can apply these funcions on the other datasets and get the outputs faster.

In [4]:
#Split dataset between features and targets, creating function

#We'll also drop the id column of the datasets, as it'll not help the model in any way.


#FUNCTION 1 - splits dataset between features and target, drops id column, splits dataset between training and testing
def features_target_split(df, t_size = 0.25):
    
    #features target split
    df = df.drop('id', axis = 1)
    df_feat = df.drop('product', axis = 1)
    df_target = df['product']
    
    #training testing split
    df_feat_train, df_feat_test, df_target_train, df_target_test = \
    train_test_split(df_feat, df_target, test_size = t_size, random_state = 1)
    
    return df_feat_train, df_feat_test, df_target_train, df_target_test

In [5]:
#FUNCTION 2 - Training linear regression model

def training_model(df_feat_train, df_feat_test, df_target_train, df_target_test):
    
    #training model
    model = LinearRegression()
    model.fit(df_feat_train, df_target_train)
    
    #testting model and looking at its scores
    predictions = model.predict(df_feat_test)
    r2 = r2_score(df_target_test, predictions)
    rmse = mean_absolute_error(df_target_test, predictions) * 0.5
    avg_pred_vol = predictions.mean()
    predictions = pd.Series(predictions)
    
    return r2, rmse, avg_pred_vol, predictions

In [6]:
def output_model_results(r2, rmse, avg_pred_vol):
        print('R2:', r2)
        print('RMSE:', rmse)
        print('Avg Predicted Volume:', avg_pred_vol)
        print()
        return None

In [7]:
#Generating datasets for models - Function 1

#geo_data_0
geo_0_feat_train, geo_0_feat_test, geo_0_target_train, geo_0_target_test = features_target_split(geo_data_0)

#geo_data_1
geo_1_feat_train, geo_1_feat_test, geo_1_target_train, geo_1_target_test = features_target_split(geo_data_1)

#geo_data_2
geo_2_feat_train, geo_2_feat_test, geo_2_target_train, geo_2_target_test = features_target_split(geo_data_2)

In [8]:
#Generating models - Function 2

#Geo0
geo0_R2, geo0_RMSE, geo0_avg_pred_vol, geo0_predictions = \
training_model(geo_0_feat_train, geo_0_feat_test, geo_0_target_train, geo_0_target_test)

In [9]:
#Geo1
geo1_R2, geo1_RMSE, geo1_avg_pred_vol, geo1_predictions = \
training_model(geo_1_feat_train, geo_1_feat_test, geo_1_target_train, geo_1_target_test)

In [10]:
#Geo2
geo2_R2, geo2_RMSE, geo2_avg_pred_vol, geo2_predictions = \
training_model(geo_2_feat_train, geo_2_feat_test, geo_2_target_train, geo_2_target_test)

In [11]:
print('GEO_0')
output_model_results(geo0_R2, geo0_RMSE, geo0_avg_pred_vol)
print('GEO_1')
output_model_results(geo1_R2, geo1_RMSE, geo1_avg_pred_vol)
print('GEO_2')
output_model_results(geo2_R2, geo2_RMSE, geo2_avg_pred_vol)

GEO_0
R2: 0.27728067218414654
RMSE: 15.540091972549451
Avg Predicted Volume: 92.49262459838863

GEO_1
R2: 0.9996221352766932
RMSE: 0.36018141262024655
Avg Predicted Volume: 69.12040524285558

GEO_2
R2: 0.20003372664683905
RMSE: 16.32527519498942
Avg Predicted Volume: 94.9568304858529



All average predicted volumes are below the needed 112 units to be profitable, but here we are considering all of the oil wells in the regions, from the test dataset. Let's select only the top 200 of each, and check those averages.

In [12]:
geo0_200 = sorted(geo0_predictions, reverse = True)
geo0_200 = np.mean(geo0_200[0:200])

print('Geo0 Top 200 mean:', round(geo0_200))

geo1_200 = sorted(geo1_predictions, reverse = True)
geo1_200 = np.mean(geo1_200[0:200])

print('Geo1 Top 200 mean:', round(geo1_200))

geo2_200 = sorted(geo2_predictions, reverse = True)
geo2_200 = np.mean(geo2_200[0:200])

print('Geo2 Top 200 mean:', round(geo2_200))

Geo0 Top 200 mean: 155
Geo1 Top 200 mean: 139
Geo2 Top 200 mean: 149


Now it is a bit of a different picture indeed - all averages are above the minimum for profit.

**Summary - Analysing Results**

Ranking the models based on their scores, we have the following:
1. GEO_1, highest R2 and lowest RMSE
2. GEO_0, second highest R2, second lowest RMSE
3. GEO_2, lowest R2, highest RMSE

Nonetheless, all models had an R2 score higher than 0, which means they are all better than average at making product predictions.

Then, based on the average predicted volume of each region, GEO_1 would be out, and the contest would be between GEO_0 and GEO_1. Let's move on to the profits calculation of the regions.

## 3 Profit Calculation

With USD 100M invested in 200 oil wells, each oil well needs to produce around 111 units of product to make it profitable -> 100M/200 = 500k -> 500k/4.5k = 111 units.

In [13]:
#Setting profit variables


budget = 100000000
oil_wells_sel = 200
unit_price = 4500

#minimum units needed for profit
min_units_profit = math.ceil(budget/oil_wells_sel/unit_price)
cost_of_development = min_units_profit*unit_price
print('Minimum units needed of produt reserve for the oil well to be profitable:', min_units_profit,'units(k barrels)')

Minimum units needed of produt reserve for the oil well to be profitable: 112 units(k barrels)


Comparing the average predicted volume per oil well in each of the regions, all of them have on average less than the needed units to be profitable. But in this case, we are looking at all the oil wells, and we only aim to develop 200 of them. 

In the next section, we'll pick only the 200 oil wells per region with the highest predicted volume of product, and base our profit calculations on that. Let's get into it now.

## 4 Profit per region, Top 200 oil wells selection, Region selection

Our goal here are as follows:
 - Create a function that calculates the predicted profit of a selected set of oil wells
 - Apply the function to the predicted values of each of the 3 regions, compare results
 - Present a suggestion based on this analysis
 
After this, we'll explore the dataset a bit further, and use boostrapping to check the confidence interval for the profits of each region. Then, reassess the situation and select the region again.

In [14]:
#FUNCTION 3 - Calculating profit per region, top 200 oil well selections
#note: input for the function must be a pd.Series data type.

def profit_calc(predicted_series):   
    sorted_series = predicted_series.sort_values(ascending = False)
    selected_wells = sorted_series[0:oil_wells_sel]
    profit_per_well = selected_wells*unit_price - cost_of_development
    total_profit = profit_per_well.sum()
    avg_profit = profit_per_well.mean()
    median_profit = profit_per_well.median()
    ROI = total_profit/budget
    
    return total_profit, avg_profit, median_profit, ROI

#Function to output the values of FUNCTION 3
def profit_calc_report(total_profit, avg_profit, median_profit, ROI):
    print('Total Profit in region, top', oil_wells_sel, 'oil wells:', round(total_profit))
    print('Average Profit per well:', round(avg_profit))
    print('Median Profit:', round(median_profit))
    print('ROI in region:', round(ROI,3))
    return None

In [15]:
predicted_series = [geo0_predictions, geo1_predictions, geo2_predictions]

i = 0

for series in predicted_series:
    print()
    print(dfs_list_names[i])
    tp, avg_p, median_p, ROI = profit_calc(series)
    profit_calc_report(tp, avg_p, median_p, ROI)
    i += 1


geo_data_0
Total Profit in region, top 200 oil wells: 38780887
Average Profit per well: 193904
Median Profit: 188658
ROI in region: 0.388

geo_data_1
Total Profit in region, top 200 oil wells: 24119384
Average Profit per well: 120597
Median Profit: 120255
ROI in region: 0.241

geo_data_2
Total Profit in region, top 200 oil wells: 33114315
Average Profit per well: 165572
Median Profit: 157298
ROI in region: 0.331


**Summary - Region Selection by Profit**

The rank of regions by highest profit is as follows:
1. Geodata 0
2. Geodata 2
3. Geodata 1

Geodata 0 has the second best model, as measured by R2, and is the region with the highest predicted profits, with the highest ROI, average and median product when compared to the other regions.

**Selected Region:** Based on the data above, we would select the region from **GeoData 0** to develop the oil wells, since it presents the highest ROI between the analyzed regions.

Now, we'll use bootstrapping to determine the profit distribution for each region, and make our assessment more certain.

## 5 Bootstrapping, Profit distribution and loss risk for each region

This is the final part of the project, where the final recommendation is to be issued.

Main objectives here are:
 - Boostrap the datasets we have to 1000 samples, each with the 200 oil wells per region.
 - Determine the average profit, the 95% confidence interval and the risk of loss.
 - Present results, suggest region for investment and reason for the choice.

In [16]:
#Getting the series we'll use

predicted_series = [geo0_predictions, 
                    geo1_predictions, 
                    geo2_predictions]

def sort_series(x):   
    sorted_series = x.sort_values(ascending = False)
    return sorted_series[0:oil_wells_sel]

geo0_pred = sort_series(geo0_predictions)*unit_price - cost_of_development
geo1_pred = sort_series(geo1_predictions)*unit_price - cost_of_development
geo2_pred = sort_series(geo2_predictions)*unit_price - cost_of_development

In [17]:
state = RandomState(12345)
def bootstrapper(data, num_samples, alpha):
    means = []
    for i in range(num_samples):
        bootstrapped_data = data.sample(frac=1, replace=True, random_state = state)
        sample_mean = bootstrapped_data.mean()
        means.append(sample_mean)
    
    upper = round(np.quantile(means, 1-alpha/2))
    lower = round(np.quantile(means, alpha/2))
    boot_mean = round(np.mean(means))
    return upper, lower, boot_mean

Now that we have our bootstrapping function ready, we'll present the following for each dataset:

 - For a Confidence Interval of 95% (alpha 5%):
      - The upper 97.5% quantile
      - The mean
      - The lower 2.5% quantile (97.5 - 2.5 = 95, our confidence interval)
      - Then calculate loss risk, based on the 500k minimum revenue for a well to be profitable

In [18]:
u0, l0, m0 = bootstrapper(geo0_pred, 1000, 0.05)
u1, l1, m1 = bootstrapper(geo1_pred, 1000, 0.05)
u2, l2, m2 = bootstrapper(geo2_pred, 1000, 0.05)

print('GEO0 Data, 95% Confidence Interval for Profits in region, per well:')
print('Upper Limit:', u0)
print('Average:', m0)
print('Lower Limit:', l0)
print()

print('GEO1 Data, 95% Confidence Interval for Profits in region, per well:')
print('Upper Limit:', u1)
print('Average:', m1)
print('Lower Limit:', l1)
print()

print('GEO2 Data, 95% Confidence Interval for Profits in region, per well:')
print('Upper Limit:', u2)
print('Average:', m2)
print('Lower Limit:', l2)
print()

GEO0 Data, 95% Confidence Interval for Profits in region, per well:
Upper Limit: 197856
Average: 193819
Lower Limit: 189883

GEO1 Data, 95% Confidence Interval for Profits in region, per well:
Upper Limit: 120792
Average: 120601
Lower Limit: 120409

GEO2 Data, 95% Confidence Interval for Profits in region, per well:
Upper Limit: 168842
Average: 165664
Lower Limit: 162585



## 6 Final Region Selection and Key Takeaways

The first point to make here is that none of the regions present a lower limit below USD 500k in the 95% confidence interval, so the risk of loss is below 2.5% in all three regions.

**Region Selection:** The suggested region for investment is still the one from **Geo Data 0**. Here's why: even after the bootstrapping exercise, this is the region with the highest average revenue per oil well. On top of that, the lower limit of the confidence interval of this regions is still above the average from the other 2 regions.